## Statsmodel Forecast Shadow Deploy Testing

Wallaroo provides a method of testing the same data against two different models or sets of models at the same time through **shadow deployments** otherwise known as **parallel deployments**.  This allows data to be submitted to a pipeline with inferences running on two different sets of models.  Typically this is performed on a model that is known to provide accurate results - the **champion** - and a model that is being tested to see if it provides more accurate or faster responses depending on the criteria known as the **challengers**.  Multiple challengers can be tested against a single champion.

Pipeline steps are set as shadow deploy steps with either the `wallaroo.pipeline.add_shadow_deploy` to create a new step, or `wallaroo.pipeline.replace_with_shadow_deploy` to replace an existing pipeline step.  All inference data is submitted to all models that are part of the shadow deploy step, and is visible through the pipeline logs and inference results when those results are in either pandas DataFrame or Apache Arrow format.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [19]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

from resources import simdb
from resources import util

pd.set_option('display.max_colwidth', None)

In [20]:
display(wallaroo.__version__)

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [21]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [22]:
# used for unique connection names

suffix='jch'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.  The models uploaded in the Upload and Deploy tutorial are referenced in this step.

In [23]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

# Get the most recent version of a model in the workspace
# Assumes that the most recent version is the first in the list of versions.
# wl.get_current_workspace().models() returns a list of models in the current workspace

def get_model(mname):
    modellist = wl.get_current_workspace().models()
    model = [m.versions()[0] for m in modellist if m.name() == mname]
    if len(model) <= 0:
        raise KeyError(f"model {mname} not found in this workspace")
    return model[0]

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
challenger01_model_name = 'forecast-challenger01-model'
challenger02_model_name = 'forecast-challenger02-model'

# upload the models

bike_day_model = get_model(control_model_name)

challenger_model_01 = get_model(challenger01_model_name)

challenger_model_02 = get_model(challenger02_model_name)


### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.  The pipeline configuration will allow for multiple replicas of the pipeline to be deployed and spooled up in the cluster.  Each pipeline replica will use 0.25 cpu and 512 Gi RAM.

In [24]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
pipeline.deploy(deployment_config = deploy_config)

{'name': 'forecast-workshop-pipeline', 'create_time': datetime.datetime(2023, 7, 27, 15, 54, 55, 416132, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': '0daa07c7-797d-429f-8a3b-7bba4f709da0', 'sha': '60156c8e9bad5b9fee1dfea2aeae05ca3f643f53b5b6d6a365e8e020c72af6ac'}]}}]"}

### Run Inference

Run a test inference to verify the pipeline is operational from the sample test data stored in `./data/testdata_dict.json`.

In [25]:
inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Replace Pipeline Step with Shadow Deploy

The pipeline step with the single model will be replaced with a shadow deploy step.  This will set the champion model as `bike_day_model`, and the two challenger steps as `challenger_model_01` and `challenger_model_02` respectively.

In [26]:
pipeline.replace_with_shadow_deploy(0, bike_day_model, 
                                    [challenger_model_01, challenger_model_02]
                                    )
pipeline.deploy()

{'name': 'forecast-workshop-pipeline', 'create_time': datetime.datetime(2023, 7, 27, 15, 54, 55, 416132, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': '0daa07c7-797d-429f-8a3b-7bba4f709da0', 'sha': '60156c8e9bad5b9fee1dfea2aeae05ca3f643f53b5b6d6a365e8e020c72af6ac'}, {'name': 'forecast-challenger01-model', 'version': '6d59e8d7-a9c9-47c4-b872-ba6c9f25f0d9', 'sha': '5e00b7106b4a07f85c14bcaaf62b05aa775448918f6446f4d834ab103dd1aa67'}, {'name': 'forecast-challenger02-model', 'version': '1a82f4f2-df82-440a-a7de-e1c82b89df7f', 'sha': 'a842d79746a91eacd1fef9bcf2de5c2d29dcb0dab0770e27ce0e402eb03f1370'}]}}, {'AuditResults': {'from': 1, 'to': None}}, {'MultiOut': {}}]"}

An inference request will now be made.  For this tutorial, the start and end time are captured to isolate this specific request in the logs.

In [27]:
import time
import datetime
# used to isolate the logs
shadow_start = datetime.datetime.now()
time.sleep(5)

inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

shadow_end = datetime.datetime.now()

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

We can see this by looking at the pipeline logs and using the `metadata` filter to retrieve the logs.

The original output is listed by `out.{variable_name}`, with the shadow outputs as `out_{model.variable_name}`.  For this example, this is `out.json`, `out_forecast-challenger01-model.json`, etc.  The command below will filter for only the time and the output variables.

In [28]:
logs = pipeline.logs(start_datetime=shadow_start, end_datetime=shadow_end)
display(logs.filter(regex = 'time|out.*'))

time                                           out.json  \
0 2023-07-27 16:33:03.694  {"forecast":[1764,1749,1743,1741,1740,1740,1740]}   

                out_forecast-challenger01-model.json  \
0  {"forecast":[1703,1757,1737,1744,1742,1743,1742]}   

                out_forecast-challenger02-model.json  
0  {"forecast":[1814,1814,1814,1814,1814,1814,1814]}

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [29]:
pipeline.undeploy()

{'name': 'forecast-workshop-pipeline', 'create_time': datetime.datetime(2023, 7, 27, 15, 54, 55, 416132, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'forecast-control-model', 'version': '0daa07c7-797d-429f-8a3b-7bba4f709da0', 'sha': '60156c8e9bad5b9fee1dfea2aeae05ca3f643f53b5b6d6a365e8e020c72af6ac'}, {'name': 'forecast-challenger01-model', 'version': '6d59e8d7-a9c9-47c4-b872-ba6c9f25f0d9', 'sha': '5e00b7106b4a07f85c14bcaaf62b05aa775448918f6446f4d834ab103dd1aa67'}, {'name': 'forecast-challenger02-model', 'version': '1a82f4f2-df82-440a-a7de-e1c82b89df7f', 'sha': 'a842d79746a91eacd1fef9bcf2de5c2d29dcb0dab0770e27ce0e402eb03f1370'}]}}, {'AuditResults': {'from': 1, 'to': None}}, {'MultiOut': {}}]"}